# Adaptive SEIR: Trace, Test and Isolate

In [ ]:
from epyc import JSONLabNotebook, ParallelLab
import epyc
import epydemic
import numpy as np
from epydemic import SEIR, SIR, ERNetwork, Monitor, ProcessSequence, rng, SynchronousDynamics
import matplotlib.pyplot as plt
from parameters import *

In [ ]:
create_data_output_directory()

## Define the Adaptive SEIR model

This Adaptive SEIR model will run simulations until model equilibrium. 

In [ ]:
# The code below was taken from a GitHub repo by Simon Dobson at https://github.com/simoninireland/introduction-to-epidemics/blob/master/src/seir.ipynb (last accessed 2023-11-09) 
# BEGIN Copied Code
class AdaptiveSEIR(SEIR):
    P_DETECT = 'pDetect'  #: Parameter for the probability of detecting an exposed neighbour of an infected node.
    P_REWIRE = 'pRewire'  #: Parameter for the probability of rewiring an SE or SI edge.

    DEFAULT_EQUILIBRIUM_STEM = SEIR.REMOVED + ".at.default.equilibrium"

    def __init__(self):
        super(AdaptiveSEIR, self).__init__()
        self._pDetect = None
        self._pRewire = None

    def build(self, params):
        super(AdaptiveSEIR, self).build(params)

        # store the parameters for later
        self._pDetect = params[self.P_DETECT]
        self._pRewire = params[self.P_REWIRE]

        self.trackNodesInCompartment(epydemic.SEIR.SUSCEPTIBLE)
        self.trackNodesInCompartment(epydemic.SEIR.REMOVED)


    def quarantine(self, n):
        g = self.network()
        rng = np.random.default_rng()
        ms = list(g.neighbors(n))
        for m in ms:
            if self.getCompartment(m) == self.SUSCEPTIBLE and rng.random() <= self._pRewire:
                # a susceptible neighbour, remove link to us
                self.removeEdge(n, m)

                # rewire to another random susceptible
                mprime = self.locus(self.SUSCEPTIBLE).draw()
                self.addEdge(m, mprime)

    def symptoms(self, t, n):
        # perform a normal becoming-symptomatic event
        super(AdaptiveSEIR, self).symptoms(t, n)

        g = self.network()
        rng = np.random.default_rng()

        # examine all neighbours and look for exposed
        # nodes to quarantine
        ms = list(g.neighbors(n))
        for m in ms:
            if self.getCompartment(m) == self.EXPOSED and rng.random() <= self._pDetect:
                # detected an exposed individual,
                # quarantine them
                self.quarantine(m)

        # quarantine the newly symptomatic node
        self.quarantine(n)
# END Copied Code

    # Override the default atEquilibrium function to end a simulation once the model reaches equilibrium.
    def atEquilibrium(self, t):
        # For SEIR models, equilibrium is reached when there are no more infected or exposed nodes, thus no more infection, removal or symptoms events.
        return len(self.compartment(self.INFECTED)) == 0 and len(self.compartment(self.EXPOSED)) == 0
    


## Define an ASEIR monitor to capture the number of removed nodes of the SEIR process at defined simulation timepoints.
Note that the Monitor class is overridden to try and reduce the memory footprint of the simulation.

In [ ]:
class ASEIR_Monitor(Monitor):
    def __init__(self):
        super().__init__()

    def observe(self, t, e):
        '''Observe the network, capturing the sizes of all loci which are then
        stored into individual time series. Another time series,
        tagged :attr:`OBSERVATIONS`, is created to store the sequence
        of times at which observations were made.

        :param t: the current simulation time
        :param e: the element (ignored)

        '''
        n = self.dynamics().locus(SEIR.REMOVED).name()
        l = self.dynamics().locus(SEIR.REMOVED)

        # if this is the first run, build the initial dicts: one
        # per locus and one for the observation times
        # (We can't do this as part of build() as it depends on which
        # other processes we're composed with, and in what order)
        if self._timeSeries is None:
            self._timeSeries = dict()
            self._timeSeries[self.OBSERVATIONS] = []
            self._timeSeries[Monitor.timeSeriesForLocus(n)] = []


        # make the observation
        self._timeSeries[self.OBSERVATIONS].append(t)
        self._timeSeries[Monitor.timeSeriesForLocus(n)].append(len(l))
               

## Define an epyc experiment to run the SEIR model with the defined parameters.

In [ ]:
def ex_1_seir(lab):
    # set the disease parameter space
    lab[AdaptiveSEIR.P_EXPOSED] = p_exposed
    lab[AdaptiveSEIR.P_INFECT_ASYMPTOMATIC] = p_infect
    lab[AdaptiveSEIR.P_INFECT_SYMPTOMATIC] = p_infect
    lab[SEIR.P_REMOVE] = p_remove
    lab[SEIR.P_SYMPTOMS] = p_remove
        
    lab[AdaptiveSEIR.P_REWIRE] = p_rewire
    lab[AdaptiveSEIR.P_DETECT] = p_detect

    # setup the monitor to capture the state of the simulation at the defult timeout. 
    lab[epydemic.Monitor.DELTA] = AdaptiveSEIR.DEFAULT_MAX_TIME

    lab['ens'] = range(ens)

    # set the topology for the generated network
    lab[ERNetwork.N] = n
    lab[ERNetwork.KMEAN] = k_mean

    # create the model, network generator, and experiment
    p = ProcessSequence([AdaptiveSEIR(), ASEIR_Monitor()])
    g = ERNetwork()
    e = epydemic.StochasticDynamics(p, g)

    # run the experiment
    lab.runExperiment(e)

## Run the experiment in parallel with the defined parameters.

Note that the controlled variable is partitioned into 100 values, and each value is run in a separate process. This aims to reduce the memory footprint of the simulation by writing the results to disk after each pdetect increment. Furthermore, this reduces memory loss in the event of a crash.

Additioanly the ensemble size is halved to reduce the memory footprint of the simulation, each set of repeats is written to a separate file. This data is then combined to perform analysis with an appropriate ensemble size.

In [ ]:
# Partition the controlled variables.
pds = np.split(p_detect, 100)

In [ ]:
ens = ens / 2

## First set of 50 experiement repeats.

In [ ]:
# Create the parallel lab.
lab = ParallelLab(JSONLabNotebook(get_out_path("ex_1_seir"), create=True), cores)

In [ ]:
# To minimise memory usage, run the experiment in parallel with a single p_detect value per process.
for i, pd in enumerate(pds):
    p_detect = pd
    lab.createWith("ex_" + str(i), ex_1_seir)

## Second set of experiement repeats.

In [ ]:
lab = ParallelLab(JSONLabNotebook(get_out_path("ex_2_seir"), create=True), cores)

for i, pd in enumerate(pds):
    p_detect = pd
    lab.createWith("ex_" + str(i), ex_1_seir)

## Further experiement to collect the point of equilibrium for each simulation.

Note that this experiment does not use the Monitor class, as this process will run the simulation until the default timeout to record the outbreak size at that point. However, this approach does not capture the equilibrium time for simulations that reach equilibrium before the timeout. Therefore, the simulation will be repeated, but only the time it takes for each simulation to reach equilibrium will be recorded.

In [ ]:
def ex_1_seir(lab):
    # set the disease parameter space
    lab[AdaptiveSEIR.P_EXPOSED] = p_exposed
    lab[AdaptiveSEIR.P_INFECT_ASYMPTOMATIC] = p_infect
    lab[AdaptiveSEIR.P_INFECT_SYMPTOMATIC] = p_infect
    lab[SEIR.P_REMOVE] = p_remove
    lab[SEIR.P_SYMPTOMS] = p_remove
        
    lab[AdaptiveSEIR.P_REWIRE] = p_rewire
    lab[AdaptiveSEIR.P_DETECT] = p_detect

    lab['ens'] = range(ens)

    # set the topology for the generated network
    lab[ERNetwork.N] = n
    lab[ERNetwork.KMEAN] = k_mean

    # create the model, network generator, and experiment
    p = AdaptiveSEIR()
    g = ERNetwork()
    e = epydemic.StochasticDynamics(p, g)

    # run the experiment
    lab.runExperiment(e)

Run experiments to collect the equilibrium time for each simulation. Note a similar approach is used to reduce the memory footprint of the simulation.

In [ ]:
lab = ParallelLab(JSONLabNotebook(get_out_path("ex_3_seir"), create=True), cores)

for i, pd in enumerate(pds):
    p_detect = pd
    lab.createWith("ex_" + str(i), ex_1_seir)

In [ ]:
lab = ParallelLab(JSONLabNotebook(get_out_path("ex_4_seir"), create=True), cores)

for i, pd in enumerate(pds):
    p_detect = pd
    lab.createWith("ex_" + str(i), ex_1_seir)